# AI解釈機能チュートリアル

このノートブックでは、**Google Gemini AI**を使用してテキスト分類結果の詳細な日本語解釈を生成する方法を説明します。

## 機能

- **AI解釈**: テンプレートを超えて、*なぜ*そのように分類されたかを説明
- **柔軟な制御**: 予測ごとにAIの使用を明示的に切り替え可能
- **エラー処理**: APIエラー時にテンプレートへフォールバック

## 目次

1. [セットアップ](#1-セットアップ)
2. [分類器の学習](#2-分類器の学習)
3. [AI要約とテンプレート要約の比較](#3-ai要約とテンプレート要約の比較)
4. [AI使用の制御](#4-ai使用の制御)
5. [直接AI解釈を取得](#5-直接ai解釈を取得)
6. [エラー処理](#6-エラー処理)

## 1. セットアップ

まず、環境をセットアップしてAIが利用可能か確認します。

In [1]:
# 依存パッケージのインストール（必要に応じて一度だけ実行）
!pip install google-genai python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.4 MB/s eta 0:00:00
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 724.2/724.2 kB 997.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 5.7 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 1.8 MB/s eta 0:00:00 0:00:01
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)

[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: pip inst

In [2]:
# .envファイルから環境変数を読み込む
# .envファイルには以下のように記載: GEMINI_API_KEY=あなたのAPIキー
from dotenv import load_dotenv
load_dotenv()

import sys
sys.path.insert(0, '..')

In [3]:
# AIが利用可能か確認
from tokusan.ai_interpreter import is_ai_available

if is_ai_available():
    print("AI解釈機能が利用可能です！")
else:
    print("AIが利用できません。")
    print("以下を確認してください：")
    print("  1. .envファイルにGEMINI_API_KEYが設定されている")
    print("  2. google-genaiがインストールされている (pip install google-genai)")

AI解釈機能が利用可能です！


## 2. 分類器の学習

フェイクニュースデータセットで分類器を学習します。

In [4]:
from tokusan import JapaneseTextClassifier
import pandas as pd

# データセットの読み込みと準備
df = pd.read_csv('fakenews.csv')
df = df.dropna(subset=['context', 'isfake'])

# 二値分類用にフィルタリング（0=本物、2=偽物）
df = df[df['isfake'].isin([0, 2])].copy()
df['label'] = (df['isfake'] == 2).astype(int)

print(f"データセットサイズ: {len(df)}")
print(f"ラベル分布:")
print(df['label'].value_counts())

データセットサイズ: 8356
ラベル分布:
label
1    4671
0    3685
Name: count, dtype: int64


In [5]:
# 分類器の作成と学習
clf = JapaneseTextClassifier(
    class_names=['Real', 'Fake'],
    random_state=42
)

result = clf.train(df['context'], df['label'])
print(result.summary_jp())

学習が完了しました。

データセット:
  - 学習データ数: 6,684
  - テストデータ数: 1,672

性能:
  - 正確度: 94.02%

クラス別メトリクス:
  - Real: 適合率=0.95, 再現率=0.92, F1=0.93
  - Fake: 適合率=0.94, 再現率=0.96, F1=0.95


## 3. AI要約とテンプレート要約の比較

AI要約とテンプレート要約の主な違いは説明の深さです。

- **テンプレート要約**: 分類結果と重要な単語を列挙
- **AI要約**: それらの単語が*なぜ*分類に影響したかを文脈を考慮して説明

In [6]:
# 分類用のサンプルテキスト
test_text = """
2020年に新型コロナウイルスが世界中で流行し、多くの国で緊急事態宣言が発令されました。
政府は感染拡大を防ぐため、外出自粛を要請しています。
専門家によると、ワクチン接種が感染予防に効果的であるとされています。
"""

# 説明付きで予測を実行
pred = clf.predict(test_text, explain=True, num_features=10)

print(f"予測クラス: {pred.predicted_class}")
print(f"確信度: {pred.confidence:.1%}")

予測クラス: Fake
確信度: 87.7%


In [7]:
# AI要約（GEMINI_API_KEYが設定されていれば自動的に使用）
print("=" * 70)
print("AI要約 (summary_jpプロパティ):")
print("=" * 70)
print(pred.summary_jp)

AI要約 (summary_jpプロパティ):
このテキストは、システムによって「Fake（偽情報）」であると87.7%の確信度で判断されました。

特に、「感染」「年」「2020」といった単語が、この文章が「Fake」であると判断される主な理由として挙げられています。システムは、過去に学習した「Fake」情報の中に、具体的な年号（特に「2020年」）や、病気の「感染」といった言葉が多く含まれているパターンを認識している可能性があります。これにより、この文章が特定の時期の感染症に関する情報であることから、「Fake」の可能性が高いと判断されたと考えられます。

一方で、「が」「に」「で」「を」といった助詞や、「し」「よる」「た」といった助動詞や接続助詞の多くは、この文章が「Fake」ではない（むしろ「Real」である）ことを示唆する要素として挙げられています。これは、文章の構造を作る上で自然な日本語の表現が使われていることを示しており、システムは文法的には比較的整った文章であると認識していることを意味します。

しかし、全体の判断としては「Fake」に傾きました。これは、文章の表面的な文法構造が整っていても、「2020年」の「感染」症に関する情報という具体的な内容が、システムが「Fake」と学習した情報パターンに強く合致したためと考えられます。特に、「専門家によると」といった表現は、一見信頼性が高そうに見えますが、AIが過去に学習したデータの中には、権威を装って誤情報を広める「Fake」の文章にこうした表現が多く使われているケースがあり、それも判断に影響した可能性があります。したがって、このテキストは、内容のキーワードが「Fake」情報の典型的なパターンに一致したため、「Fake」と判断されたと言えるでしょう。


In [8]:
# 比較用にテンプレート要約を確認
# プライベートメソッドで直接アクセス可能
print("=" * 70)
print("テンプレート要約（比較用）:")
print("=" * 70)
print(pred._template_summary_jp())

テンプレート要約（比較用）:
予測結果: Fake (87.7%の確率)
クラス確率: Real: 12.3%, Fake: 87.7%

説明:
このインスタンスは0.877対0.123でFakeと分類されました。Fakeへの分類に最も強い影響を与えた言葉は感染, 年, 2020で、それぞれの重み=+0.050（中）, 重み=+0.046（弱）, 重み=+0.043（弱）となっています。
他にFakeへの分類の確率を上げた言葉としてが (重み=-0.037（弱）)、に (重み=-0.033（弱）)、で (重み=-0.032（弱）)などが挙げられます。Realへの分類への確率を上げた言葉として、が (重み=+0.037（弱）)、に (重み=+0.033（弱）)、で (重み=+0.032（弱）)などが挙げられます。


### 主な違い

| 項目 | テンプレート要約 | AI要約 |
|------|-----------------|--------|
| 内容 | 確率と単語を列挙 | 理由と文脈を説明 |
| 深さ | 表面的な事実 | 深い解釈 |
| 速度 | 即座 | 1〜2秒（API呼び出し） |
| コスト | 無料 | Gemini APIクォータを使用 |

## 4. AI使用の制御

要約にAIを使用するかどうかを明示的に制御できます。

In [9]:
# テンプレート要約を強制（AIを使用しない）
pred_no_ai = clf.predict(test_text, explain=True, use_ai=False)

print("use_ai=Falseの場合:")
print(pred_no_ai.summary_jp)

use_ai=Falseの場合:
予測結果: Fake (87.7%の確率)
クラス確率: Real: 12.3%, Fake: 87.7%

説明:
このインスタンスは0.877対0.123でFakeと分類されました。Fakeへの分類に最も強い影響を与えた言葉は年, 感染, 2020で、それぞれの重み=+0.045（弱）, 重み=+0.044（弱）, 重み=+0.037（弱）となっています。
他にFakeへの分類の確率を上げた言葉としてが (重み=-0.033（弱）)、で (重み=-0.029（弱）)、に (重み=-0.028（弱）)などが挙げられます。Realへの分類への確率を上げた言葉として、が (重み=+0.033（弱）)、で (重み=+0.029（弱）)、に (重み=+0.028（弱）)などが挙げられます。


In [10]:
# AI要約を強制（利用不可の場合はエラー）
pred_with_ai = clf.predict(test_text, explain=True, use_ai=True)

print("use_ai=Trueの場合:")
print(pred_with_ai.summary_jp)

use_ai=Trueの場合:
このテキストは、AIによって「フェイク（偽情報）」であると、約87.7%という非常に高い確信度で判断されました。

この判断に影響を与えた単語を見てみましょう。「年」「感染」「2020」といった単語は、このテキストが「フェイク」であると判断するのを強く後押ししました。特に「2020」や「感染」は、新型コロナウイルスが流行し始めた時期とテーマを直接指す言葉です。AIが過去に学習したデータの中に、これらのキーワードが、事実ではない情報や誤情報（フェイクニュース）とセットで頻繁に登場していたため、AIは「これらの単語が多いとフェイクらしい」と判断したと考えられます。

一方で、「が」「で」「し」「に」といった、一般的な日本語の文章で使われる助詞や接続詞などは、むしろ「フェイクではない（真実らしい）」という判断を少し支持しています。これは、この文章が文法的に自然で、不自然な表現が少ないことを示唆しています。

総合的に見ると、入力されたテキストは、2020年の新型コロナウイルスの流行、緊急事態宣言、外出自粛、ワクチン接種といった、実際に起こった出来事を記述しています。しかしAIは、文章の内容そのものの真偽を直接判断しているわけではありません。AIは、学習したデータに基づいて、文章中の特定のキーワードの組み合わせや、文章が持つパターンが、過去のフェイクニュースとどれだけ似ているかを見て判断します。このテキストの場合、新型コロナウイルス感染症に関連する「2020年」や「感染」といった特定のキーワードが、AIが過去に「フェイク」と学習した情報群と強く関連付けられていたため、内容が事実に基づいていたとしても、「フェイク」という判断が下された可能性が高いと言えるでしょう。


In [11]:
# 自動検出（デフォルト）- GEMINI_API_KEYが設定されていればAIを使用
pred_auto = clf.predict(test_text, explain=True)  # use_ai=Noneがデフォルト

print("use_ai=None（自動検出）の場合:")
print(f"AIが使用されました: {is_ai_available()}")

use_ai=None（自動検出）の場合:
AIが使用されました: True


## 5. 直接AI解釈を取得

`ExplanationResult`オブジェクトから直接AI解釈を取得することもできます。

In [12]:
# 説明付きで予測を取得
pred = clf.predict(test_text, explain=True, use_ai=False)  # summary_jpにはAIを使用しない

# explanationオブジェクトにアクセス
explanation = pred.explanation

print("単語の重み:")
for word, weight in explanation.word_weights[:5]:
    sign = "+" if weight > 0 else ""
    print(f"  {word}: {sign}{weight:.4f}")

単語の重み:
  感染: +0.0446
  が: -0.0392
  で: -0.0346
  年: +0.0346
  2020: +0.0338


In [13]:
# ExplanationResultから直接AI解釈を取得
ai_interpretation = explanation.get_ai_interpretation()

print("=" * 70)
print("直接AI解釈:")
print("=" * 70)
print(ai_interpretation)

直接AI解釈:
このテキストは、87.7%という高い確信度で「Fake（偽情報）」であると判断されました。

この判断に影響を与えた単語を見てみると、「感染」「年」「2020」といった言葉が「Fake」である可能性を高める要因として挙げられています。特に「2020」や「感染」は、新型コロナウイルスが世界中で流行し始めた時期や状況を示す具体的な単語です。一見すると客観的な事実のように思えますが、この分類システムは、特定の時期に広まった一般的な情報でありながら、その情報源が不明確であったり、詳細な根拠が示されていなかったりする文章を「Fake」と学習している可能性があります。つまり、誰でも知っているような情報が淡々と述べられているだけで、具体的な裏付けがない点が、偽情報の特徴と似ていると判断されたと考えられます。「ます」という丁寧語の終助詞も「Fake」の判断を後押ししていますが、これは偽情報がしばしば読者に安心感を与えるために丁寧な言葉遣いをすることがあるため、そのパターンを学習したのかもしれません。

一方で、「が」「で」「に」「し」「を」「た」といった助詞や動詞の活用形を示す単語は、「Fake」の判断を打ち消し、「Real（本物）」である可能性を示唆していました。これらの言葉は文章の構造やリズムを作る上で非常に重要です。システムは、このテキストで使われている助詞や活用形のパターンが、信頼できる情報源から発信される「Real」な文章の一般的なパターンと一致する部分もある、と捉えた可能性があります。

しかし、総合的に見ると、このテキストは新型コロナウイルスに関する一般的な情報を列挙しているものの、その情報がどこから来たのか、誰が言っているのかという具体的な情報源が一切記されていません。例えば、「専門家によると」とありますが、その専門家が誰なのか、どのような機関の専門家なのかといった詳細が不明です。このような情報源の曖昧さや、具体的な根拠の提示がない点が、システムが学習した「Fake」情報のパターンと強く合致したため、最終的に「Fake」という判断に至ったと考えられます。内容そのものが完全に誤りであるというよりも、その提示の仕方や情報の根拠が不足している点が、信頼性の低い情報とみなされた大きな理由と言えるでしょう。


In [14]:
# フォールバックオプション付き
# AIが失敗した場合、エラーを発生させずにテンプレート要約を返す
interpretation = explanation.get_ai_interpretation(fallback_to_template=True)
print(interpretation)

このテキストは、AI（人工知能）モデルによって87.7%という高い確信度で「Fake（偽情報）」であると判断されました。

特に「感染」「年」「2020」といった単語が、このテキストを「Fake」と判断する上で重要な手がかりとなりました。AIは、「2020年」という具体的な時期の指定や、「感染」という言葉が、過去に流布された偽情報や誤解を招く情報の中で頻繁に使われていたパターンを学習していると考えられます。また、「ます」のような丁寧語の語尾も、特定の情報源や伝え方と関連付けられた可能性があります。

一方で、「が」「で」「に」「し」「を」「た」といった、ごく一般的な助詞や動詞の活用語尾は、AIが「Fake」ではない（つまり「Real」である）と判断する傾向があることを示しています。これは、AIが「Real」な情報と判断するテキスト、例えば一般的なニュース記事などでは、これらの助詞がより自然な形で頻繁に使われていると学習したためと考えられます。

このテキスト自体は、新型コロナウイルスに関する一般的な事実を述べているように見えます。しかし、AIは表面的な内容だけでなく、文章の「スタイル」や「構成」を重視して判断しています。具体的には、「2020年」という過去の具体的な時点での出来事を、改めて事実として語る文章の形が、AIが過去に学習した「Fake」情報のパターンと合致したと考えられます。偽情報はしばしば、まず確かな事実を提示し、そこから誤った情報へと誘導する手法を用いることがあります。そのため、このテキストの導入部分の書き方が、AIにはその種の偽情報と似ていると映ったのかもしれません。AIは、単語の意味そのものよりも、それらがどのような文脈で、どのような頻度や組み合わせで使われるかという、より深いパターンを読み取って判断していると言えます。


## 6. エラー処理

AI統合はフォールバックオプション付きの堅牢なエラー処理を提供します。

In [15]:
from tokusan.exceptions import AIInterpretationError

# 例：AIエラーを適切に処理
try:
    pred = clf.predict(test_text, explain=True, use_ai=True, fallback_to_template=False)
    print(pred.summary_jp)
except AIInterpretationError as e:
    print(f"AIエラー: {e}")
    # 手動でテンプレートにフォールバック
    print("\nテンプレートにフォールバック...")
    print(pred._template_summary_jp())

このテキストは、「Fake」（偽情報）であると、87.7%という高い確信度で判定されました。

この判定に特に強く影響したのは、「感染」「年」「2020」といった単語です。AI（人工知能）は、過去の学習データから、これらの言葉が特定の時期（2020年）の「感染症」に関する情報で使われていると、「Fake」である可能性が高いと判断する傾向があるようです。これは、2020年の新型コロナウイルス流行時には、多くの誤った情報や憶測が飛び交ったため、それらの「Fake」ニュースの中にこれらのキーワードが頻繁に含まれていたことをAIが学習した結果だと考えられます。

一方で、「が」「で」「を」「に」「し」「た」「よる」といった助詞や助動詞（言葉のつながりをスムーズにする役割の言葉）は、むしろ「Fake」であることに反対し、このテキストを「Real」（本物の情報）に近づける方向に働きました。これらの言葉は日本語の文章でごく一般的に使われるものであり、AIは「Real」と判断されるような信頼できる情報源のテキストでは、これらの助詞が自然に使われていることを学習しているためです。このことから、このテキスト自体が極端に不自然な文章構造をしているわけではないことが伺えます。

総合的に見ると、このテキストの内容は「2020年の新型コロナウイルス流行に関する一般的な事実」を述べているように見えます。しかし、AIの判定では、内容の真偽そのものよりも、「2020年」「感染」といった、まさにその時期の「新型コロナウイルス関連情報」を指し示すキーワードが強く影響したと推測されます。つまり、AIは「2020年の新型コロナウイルスに関する情報は、過去に誤情報が非常に多く発生したトピックである」という傾向を学習しており、このテキストがそのパターンに強く合致したため、「Fake」と判定した可能性が高いのです。文章自体の文法的な不自然さよりも、特定のテーマや時代背景を示すキーワードが、過去の「Fake」情報のデータと強く結びついたことが、今回の判定の大きな要因と考えられます。


In [16]:
# 推奨：本番環境ではfallback_to_template=True（デフォルト）を使用
pred = clf.predict(
    test_text,
    explain=True,
    use_ai=True,
    fallback_to_template=True  # デフォルト - 常に何かを返す
)

# これはエラーを発生させない - AIが失敗したらテンプレートにフォールバック
print(pred.summary_jp)

このテキストは、機械学習モデルによって「Fake（偽情報）」であると87.7%の高い確信度で判断されました。

まず、このテキストが「Fake」だと判断される要因となった単語について見てみましょう。「年」「感染」「2020」「ます」といった言葉が、テキストが偽情報である可能性を高めていると分析されました。特に「2020年」という具体的な数字があるにもかかわらず、全体的に「世界中で流行」「多くの国で緊急事態宣言」といった漠然とした表現が多く、具体的な情報に乏しい点が、機械学習モデルによって偽情報の特徴として捉えられたと考えられます。

一方で、「が」「し」「を」「で」「に」「た」といった助詞や助動詞は、「Fake」ではない、つまり「Real（本物の情報）」である可能性を示唆する要因として分析されました。これらの単語自体に特別な意味はありませんが、一般的な本物の文章では、より複雑で多様な文のつながりが使われ、これらの助詞が自然に多く現れる傾向があるため、Realな文章のパターンの一部として認識されたと考えられます。しかし、これらの要因は、前述の「Fake」を支持する単語の重みと比べると影響が小さく、テキスト全体の判断を覆すまでには至りませんでした。

総合的に見ると、このテキストは新型コロナウイルスに関するもっともらしい情報が並べられていますが、「専門家によると」という情報源の不明瞭さや、「世界中で流行」「多くの国で緊急事態宣言」といった、誰もが知っているような一般的な事実や常識を抽象的に述べているだけで、具体的な情報（例えば、特定の国の詳細な対策、具体的なデータ、専門家の所属や発言内容など）が一切含まれていないという特徴があります。機械学習モデルは、このような「具体的な根拠や詳細を欠き、一般的な情報や常識を並べ立てているだけ」という特徴を、偽情報のパターンとして学習しているため、このテキストを「Fake」と判断したと考えられます。あたかも事実であるかのように装いながらも、その実体が伴わない情報であると見抜かれたのです。


## まとめ

### クイックリファレンス

```python
# .envからAPIキーを読み込む
from dotenv import load_dotenv
load_dotenv()

from tokusan import JapaneseTextClassifier
from tokusan.ai_interpreter import is_ai_available

# AI利用可能か確認
if is_ai_available():
    print("AIは準備完了！")

# AI要約付きで予測（自動）
pred = clf.predict(text, explain=True)
print(pred.summary_jp)  # 利用可能ならAIを使用

# テンプレートを強制（AIなし）
pred = clf.predict(text, explain=True, use_ai=False)

# AIを強制（利用不可ならエラー）
pred = clf.predict(text, explain=True, use_ai=True)

# 直接AI解釈を取得
interpretation = pred.explanation.get_ai_interpretation()
```

### パラメータ

| パラメータ | 型 | デフォルト | 説明 |
|-----------|------|---------|-------------|
| `use_ai` | `bool/None` | `None` | `None`=自動、`True`=AI強制、`False`=テンプレート強制 |
| `fallback_to_template` | `bool` | `True` | AIエラー時にエラーを発生させずテンプレートを返す |

### セットアップチェックリスト

1. `.env`ファイルに`GEMINI_API_KEY=あなたのキー`を記載
2. インストール: `pip install google-genai python-dotenv`
3. tokusanをインポートする前に`load_dotenv()`を呼び出す
4. `is_ai_available()`で確認